In [3]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="./processed_human/merged.json")
#dataset = load_dataset("processed_human/merged")
dataset["train"][50]

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1720.39it/s]


Dataset json downloaded and prepared to /home/soma1/.cache/huggingface/datasets/json/default-00ac06e086774f7d/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 882.45it/s]


{'label': '12', 'text': '재미있는 낱자'}

In [4]:
from transformers import AutoTokenizer

HUGGINGFACE_MODEL_PATH = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8934
    })
})

In [10]:
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=31).select(range(1000))
#small_train_dataset = tokenized_datasets["test"].shuffle(seed=31).select(range(1000))

seed = 42  # Set the seed value
tokenized_datasets = tokenized_datasets.shuffle(seed=seed)

train_ratio = 0.8  # Specify the ratio of the training set
val_ratio = 0.1  # Specify the ratio of the validation (evaluation) set
test_ratio = 0.1  # Specify the ratio of the test set

num_examples = len(tokenized_datasets["train"])  # Get the total number of examples
train_size = int(train_ratio * num_examples)  # Calculate the size of the training set
val_size = int(val_ratio * num_examples)  # Calculate the size of the validation (evaluation) set

train_indices = list(range(train_size))  # Get the indices for the training set
val_indices = list(range(train_size, train_size + val_size))  # Get the indices for the validation (evaluation) set
test_indices = list(range(train_size + val_size, num_examples))  # Get the indices for the test set

train_dataset = tokenized_datasets["train"].select(train_indices)
val_dataset = tokenized_datasets["train"].select(val_indices)
test_dataset = tokenized_datasets["train"].select(test_indices)


In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels = 3)


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initial

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir = "./_0714_hyperparameters")

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`